In [134]:
import json
import pickle
from pprint import pprint as pp
from collections import defaultdict as dd, OrderedDict as od
import re

In [135]:
with open('product_attributes_taxonomy.txt', 'r', encoding='utf-8') as f:
    taxonomy_text = f.read()

In [175]:
taxonomy = od()
prev_tab_level = 0
parent = ''
prev_line = ''
path = []
for line in taxonomy_text.split('\n'):
    line = line.strip(' ')
    tab_level = line.count('\t')
    if tab_level > prev_tab_level:
        path.append(parent)    
        parent = prev_line.split()[0]
    elif tab_level < prev_tab_level:
        parent = path[tab_level - prev_tab_level]
        path = path[:tab_level - prev_tab_level]
    for word in line.strip('\t').split(' '):
        if not tab_level == 0:
            taxonomy[tuple(word.lower().split('-'))] = path[1:] + [parent]
        else:
            taxonomy[tuple(word.lower().split('-'))] = [line.split()[0]]
    prev_line = line
    prev_tab_level = tab_level
pp((taxonomy))

OrderedDict([(('screen',), ['SCREEN']),
             (('display',), ['SCREEN']),
             (('type',), ['SCREEN', 'DISPLAY']),
             (('led',), ['SCREEN', 'DISPLAY', 'TYPE']),
             (('oled',), ['SCREEN', 'DISPLAY', 'TYPE', 'LED']),
             (('lcd',), ['SCREEN', 'DISPLAY', 'TYPE']),
             (('tft',), ['SCREEN', 'DISPLAY', 'TYPE', 'LCD']),
             (('ips',), ['SCREEN', 'DISPLAY', 'TYPE', 'LCD']),
             (('resolution',), ['SCREEN', 'DISPLAY']),
             (('960x640',), ['SCREEN', 'DISPLAY', 'RESOLUTION']),
             (('1136x640',), ['SCREEN', 'DISPLAY', 'RESOLUTION']),
             (('1335x750',), ['SCREEN', 'DISPLAY', 'RESOLUTION']),
             (('1920x1080',), ['SCREEN', 'DISPLAY', 'RESOLUTION']),
             (('1048x768',), ['SCREEN', 'DISPLAY', 'RESOLUTION']),
             (('2048x1535',), ['SCREEN', 'DISPLAY', 'RESOLUTION']),
             (('inch', 'size'), ['SCREEN', 'DISPLAY']),
             (('3,5',), ['SCREEN', 'DISPLAY', 'INCH-SI

In [177]:
with open('product_attributes_taxonomy.pickle', 'wb') as f:
    f.write(pickle.dumps(taxonomy))

In [166]:
def make_regex(taxonomy_dict):
    regex = ''
    for entry in sorted(taxonomy_dict.keys(), key=lambda x: (x[0], -len(x))):
        regex += '[ -]+'.join(entry) + '|'
    return re.compile(regex.strip('|'))

regex = make_regex(taxonomy)
regex


re.compile(r'1048x768|1080p|1136x640|1335x750|140p|16|160|16:10|16:9|1920x1080|2048x1535|3,5|32|320p|330|3:2|4|4.7|400|480p|4:3|4k|5.5|5k|64|720p|8|960x640|accelerometer|aluminum|amplifier|aperture|app[ -]+launch[ -]+time|artefacts|aspect[ -]+ratio|auto[ -]+focus|bandwidth|battery[ -]+capacity|battery[ -]+life|battery[ -]+drain|battery[ -]+consumption|battery|blur|brightness|build[ -]+quality|camera|capacity|ceramic|color[ -]+accuracy|color|compass|compression|contrast|cores|cpu|design|display|durability|dust[ -]+resistance|dynamic[ -]+range|exposure|fingerprint|flash|frequency|glass|gorilla[ -]+glass[ -]+1|gorilla[ -]+glass[ -]+2|gorilla[ -]+glass[ -]+3|gorilla[ -]+glass[ -]+4|gpu|gyroscope|hdr|inch[ -]+size|ios|ip67|ip68|ips|lag|lcd|led|lens|light[ -]+sensibility|low[ -]+light[ -]+perfomance|magnesium|material|maximum[ -]+picture[ -]+resolution|maximum[ -]+video[ -]+resolution|megapixels|memory|mono|multitasking|noise|ois|oled|optical[ -]+image[ -]+stabilization|os|pentile|pereferia|

In [172]:
regex.findall('The battery life and the picture quality of the camera sensor as well as the display ppi')

['battery life', 'picture quality', 'camera', 'sensor', 'display', 'ppi']

In [176]:
def get_taxonomy_paths(text, regex, taxonomy):
    taxonomy_paths = []
    search_results = regex.findall(text)
    for sr in search_results:
        taxonomy_paths.append(taxonomy[tuple(re.split('[- ]+', sr))])
    return taxonomy_paths

get_taxonomy_paths('The battery life and the picture quality of the camera sensor as well as the display ppi', regex, taxonomy)

[['BATTERY'],
 ['CAMERA'],
 ['CAMERA'],
 ['CAMERA'],
 ['SCREEN'],
 ['SCREEN', 'DISPLAY', 'PIXEL-DENSITY']]